In [ ]:
# import the necessary packages
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import math

import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization

from tensorflow.keras.optimizers import SGD, RMSprop, Adam


np.random.seed(1000)

In [ ]:
def generate_data(N, M, batch_size):
    a = np.array(range(pow(2, N)))
    K = np.tile(a, batch_size)
    data = K

    a = np.eye(modulation_order)
    K = np.tile(a, batch_size)
    label = np.transpose(K)
    return data, label


def generate(M, N, batch_size):
    data,label = generate_data(N, M, batch_size)

    ran1 = np.random.permutation(batch_size * pow(2, N))
    ran2 = np.random.permutation(batch_size * pow(2, N))

    symbol1 = 2 * data[ran1] - 1
    symbol2 = 2 * data[ran2] - 1

    SPC = math.sqrt(0.8) * symbol1 +math.sqrt(0.2) * symbol2

    label1 = label[ran1,:].astype('float32')
    label2 = label[ran2,:].astype('float32')

    return SPC, label1, label2


def generate_input(H1_real, H1_image, SPC, N, batch_size, sigma):

    N_real, N_image = generate_channel(N, batch_size * pow(2, N), 0)

    input1_real = H1_real * SPC + sigma * N_real
    input1_img = H1_image * SPC + sigma * N_image

    input1 = np.transpose(np.concatenate((input1_real, input1_img), axis=0))

    return input1


def generate_channel(N, M, k):
    h1 = np.random.randn(N, M) / math.sqrt(2)
    h2 = np.random.randn(N, M) / math.sqrt(2)

    if k == 0:
        return h1, h2
    else:
        return 2 * h1, 2 * h2


In [ ]:
def model_alex_cnn():
    #Instantiate an empty model
    model = Sequential()

    # 1st Convolutional Layer
    model.add(Conv2D(filters=96, input_shape=(224,224,3), kernel_size=(11,11), strides=(4,4), padding='valid'))
    model.add(Activation('relu'))
    # Max Pooling
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))

    # 2nd Convolutional Layer
    model.add(Conv2D(filters=256, kernel_size=(11,11), strides=(1,1), padding='valid'))
    model.add(Activation('relu'))
    # Max Pooling
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))

    # 3rd Convolutional Layer
    model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
    model.add(Activation('relu'))

    # 4th Convolutional Layer
    model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
    model.add(Activation('relu'))

    # 5th Convolutional Layer
    model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='valid'))
    model.add(Activation('relu'))
    # Max Pooling
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))

    # Passing it to a Fully Connected layer
    model.add(Flatten())
    # 1st Fully Connected Layer
    model.add(Dense(4096, input_shape=(224*224*3,)))
    model.add(Activation('relu'))
    # Add Dropout to prevent overfitting
    model.add(Dropout(0.4))

    # 2nd Fully Connected Layer
    model.add(Dense(4096))
    model.add(Activation('relu'))
    # Add Dropout
    model.add(Dropout(0.4))

    # 3rd Fully Connected Layer
    model.add(Dense(1000))
    model.add(Activation('relu'))
    # Add Dropout
    model.add(Dropout(0.4))

    # Output Layer
    model.add(Dense(modulation_order))
    model.add(Activation('sigmoid'))

    return model

In [ ]:
def my_loss(y_t , y_p) :
    m_loss = tf.reduce_mean(tf.compat.v2.nn.softmax_cross_entropy_with_logits(logits=y_p, labels=y_t))
    return m_loss

def my_acc(y_t, y_p):
    print(y_p.shape)
    print(y_t.shape)
    K.print_tensor(y_t)
    K.print_tensor(y_p)
    correct_prediction = tf.equal(tf.argmax(y_p, axis= 1), tf.argmax(y_t, axis= 1))
    acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    return acc

In [ ]:
# network and training
VERBOSE = "auto"
OPTIMIZER = Adam(epsilon= 5e-04)
VALIDATION_SPLIT=0.3
N = 1
M = 1
n_symbol = 100
n_iteration = 1
NB_EPOCH = 3
modulation_order = 2

SNR_db = np.array(list(range(0, 11, 2)))
# SNR_db = np.array(list(range(100,101)))
batch_size = 224*224*3
test_size = 0.5

ERROR_user1 = np.zeros([len(SNR_db), n_iteration])
ERROR_user2 = np.zeros([len(SNR_db), n_iteration])

In [ ]:
for k in range(n_iteration):
    H1_real, H1_image = generate_channel(N, M, 0)
    H2_real, H2_image = generate_channel(N, M, 1)
    print ('training iteration %d' %(k))

    for i in range(len(SNR_db)):

        SPC, label1, _ = generate(M, N,  batch_size)
        signal_power = np.mean(pow(SPC, 2))
        sigma = math.sqrt(signal_power / (math.sqrt(N) * pow(10, SNR_db[i] / 10)))
        input1_train = generate_input(H1_real, H1_image, SPC, N,  batch_size, sigma)
        input1_train = tf.reshape(input1_train, [-1, 224, 224, 3])
        # initialize the optimizer and model
        model_d1 = model_alex_cnn()
        model_d1.compile(loss=my_loss, optimizer=OPTIMIZER,metrics=[my_acc])
        history_d1 = model_d1.fit(input1_train, label1, epochs=NB_EPOCH,verbose=VERBOSE ,validation_split=VALIDATION_SPLIT )


    for i in range(len(SNR_db)):

        SPC, _, label2 = generate(M, N,  batch_size)
        signal_power = np.mean(pow(SPC, 2))
        sigma = math.sqrt(signal_power / (math.sqrt(N) * pow(10, SNR_db[i] / 10)))
        input2_train = generate_input(H2_real, H2_image, SPC, N, batch_size, sigma)
        input2_train = tf.reshape(input2_train, [-1, 224, 224, 3])
        # initialize the optimizer and model
        model_d2 = model_alex_cnn()
        model_d2.compile(loss=my_loss, optimizer=OPTIMIZER,metrics=[my_acc])
        history_d2 = model_d2.fit(input2_train, label2, epochs=NB_EPOCH,verbose=VERBOSE ,validation_split=VALIDATION_SPLIT )


training iteration 0
Epoch 1/3
(None, 2)
(None, 2)
(None, 2)
(None, 2)
 [[0 1]
 [1 0]]
 [[0.497646451 0.487714976]
 [0.499586731 0.482378244]]
1/1 [==============================] - ETA: 0s - loss: 0.6914 - my_acc: 0.5000(None, 2)
(None, 2)
 [[1 0]
 [0 1]]
 [[0.496516436 0.497038633]
 [0.496754646 0.4976466]]
1/1 [==============================] - 2s 2s/step - loss: 0.6914 - my_acc: 0.5000 - val_loss: 0.6931 - val_my_acc: 0.5000
Epoch 2/3
 [[1 0]
 [0 1]]
 [[0.505122185 0.49357608]
 [0.475014955 0.505484045]]
1/1 [==============================] - ETA: 0s - loss: 0.6827 - my_acc: 1.0000 [[1 0]
 [0 1]]
 [[0.494004428 0.497782975]
 [0.493771911 0.498518854]]
1/1 [==============================] - 1s 541ms/step - loss: 0.6827 - my_acc: 1.0000 - val_loss: 0.6929 - val_my_acc: 0.5000
Epoch 3/3
 [[0 1]
 [1 0]]
 [[0.483868599 0.477304399]
 [0.48965475 0.508219719]]
1/1 [==============================] - ETA: 0s - loss: 0.6995 - my_acc: 0.0000e+00 [[1 0]
 [0 1]]
 [[0.493085295 0.498766]
 [0.493

In [ ]:
for k in range(n_iteration):

    print('testing operation %d'%(k))
    for i in range(len(SNR_db)):

        SPC_test, test_label1, test_label2 = generate(M, N, int(batch_size * test_size ) )
        signal_power = np.mean(pow(SPC, 2))
        sigma = math.sqrt(signal_power / (math.sqrt(N) * pow(10, SNR_db[i] / 10)))

        input1_test = generate_input(H1_real, H1_image, SPC_test, N, int(batch_size * test_size), sigma)
        score1 = model_d1.evaluate(input1_test,test_label1, verbose=VERBOSE)
        ERROR_user1[i,k] = 1-score1[1]


        input2_test = generate_input(H2_real, H2_image, SPC_test, N, int(batch_size * test_size), sigma)
        score2 = model_d2.evaluate(input2_test,test_label2, verbose=VERBOSE)
        ERROR_user2[i,k] = 1-score2[1]


error1 = np.mean((ERROR_user1),axis=1)
error2 = np.mean((ERROR_user2),axis=1)


testing operation 0


ValueError: ignored

In [ ]:
plt.figure()
plt.semilogy(SNR_db,error1, ls = '--', marker = 'o',label='user1')
plt.semilogy(SNR_db,error2, ls = '--', marker = '+',label='user2')
plt.grid()
plt.legend()
plt.ylim(pow(10,-6),pow(10,0) )
plt.xlabel('SNR')
plt.ylabel('SER')
plt.title('SER of user2 in SISO_NOMA BPSK_DNN')
plt.savefig('res1')
plt.show()

print(error1)
print(error2)

In [ ]:
# list all data in history
# summarize history for accuracy
plt.figure()
plt.plot(history_d1.history['my_acc'])
plt.plot(history_d1.history['val_my_acc'])
plt.title('model-1 accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train UE1', 'test UE1'], loc='upper right')
plt.show()

In [ ]:
# list all data in history
# summarize history for accuracy
plt.figure()
plt.plot(history_d2.history['my_acc'])
plt.plot(history_d2.history['val_my_acc'])
plt.title('model-2 accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train UE2', 'test UE2'], loc='upper right')
plt.show()

In [ ]:
# summarize history for loss
plt.figure()
plt.plot(history_d1.history['loss'])
plt.plot(history_d1.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train UE1', 'test UE1'], loc='upper right')
plt.show()

In [ ]:
# summarize history for loss
plt.figure()
plt.plot(history_d2.history['loss'])
plt.plot(history_d2.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train UE2', 'test UE2'], loc='upper right')
plt.show()

In [ ]:
print(input1_train.shape)
print(input2_train.shape)
print(input1_test.shape)
print(input2_test.shape)

print(label1.shape)
print(label2.shape)
print(test_label1.shape)
print(test_label2.shape)

print(input1_train[111])
print(label1[111])

In [ ]:
model_d1.save("model_dnn_UE1")
model_d2.save("mode2_dnn_UE2")